<a href="https://colab.research.google.com/github/m-mehabadi/grad-maker/blob/main/_notebooks/Testing_GradientMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://www.cs.cmu.edu/~pradeepr/convexopt/Lecture_Slides/dual-ascent.pdf

In [ ]:
import numpy as np
# import torch
import matplotlib.pyplot as plt

In [ ]:
# dim=2000000
# a = 100*np.random.randn(4, dim) + 1123
# g = np.random.randn(dim, 1)
# b = (a-np.mean(a, axis=1)[:, np.newaxis])/np.std(a, axis=1)[:, np.newaxis]
# np.mean(b@g/dim)

In [ ]:
# def gradient_maker(domain_grads, epsilon=0.5, alpha=0.01, eff=0.1):
#     ###### NOTES:
#     ### make sure that `domain_grads` dtype is float precision not integer or long.

#     def log():
#         # print(f"g={g}")
#         print(f"Iter={iter}, Condition={np.min(dgr@g/dim)}")
    
#     def null(A, eps=1e-15):
#         u, s, vh = np.linalg.svd(A)
#         # print(vh)
#         null_space = np.compress(s <= eps, vh, axis=0)
#         return null_space.T

#     def null(A):
#         from scipy.linalg import null_space
#         n = null_space(A, rcond=1)
#         if n.shape[1] < 1:
#             return False, None
#         r = np.sqrt(np.sum((A@n)**2, axis=0)/n.shape[1])
#         if np.min(r) >= 1e-2:
#             return False, None
#         return True, n.T[np.argmin(r)].reshape(-1, 1)
    
#     def scale():
#         G = np.concatenate((g.reshape(1, -1), domain_grads), axis=0)
#         is_fine, n = null(G@G.T)
#         if not is_fine:
#             return np.mean(np.sqrt(np.sum(domain_grads**2, axis=1)))/np.sqrt(np.sum(g**2))
#         # print(n)
#         n /= np.sum(n[1:])
#         # print(n)
#         return np.abs(n[0,0])

#     dgr = np.copy(domain_grads)
#     number_of_domains, dim = dgr.shape

#     #
#     dgr /= np.sqrt(np.sum(dgr**2, axis=1))[:, np.newaxis]
    
#     #
#     g = np.random.randn(dim)
#     g /= np.sqrt(np.sum(g**2))
#     u_ = np.zeros(number_of_domains)
    
#     iter = 0
#     while not np.min(dgr@g/dim) >= epsilon:

#         log()
        
#         u_ = u_ + alpha*((1.+eff)*epsilon - (dgr@g)/dim)
#         g = (1./number_of_domains)*np.sum(((1+u_).reshape(number_of_domains, 1))*dgr, axis=0)

#         iter += 1
    
#     log()
#     return scale()*g

### Testcases

Now let's write some test cases to make sure everything is working correctly


In [ ]:
epsilon=10
alpha=0.9

In [ ]:
grads = 100*np.random.randn(20, 10000000) - 10
# grads = np.array([[1., 0], [1, 5]])#, [0, 1]])
# print(grads)

In [ ]:
# AA = grads
# AA_g = g

# px = np.array([0, AA[0][0].item()])
# py = np.array([0, AA[0][1].item()])

# px2 = np.array([0, AA[1][0].item()])
# py2 = np.array([0, AA[1][1].item()])

# px3 = np.array([0, AA_g[0].item()])
# py3 = np.array([0, AA_g[1].item()])

# plt.plot(px, py, 'blue')
# plt.plot(px2, py2, 'blue')
# plt.plot(px3, py3, 'green')
# plt.show()

In [ ]:
g = gradient_maker(grads, epsilon=epsilon, alpha=alpha)
# print(g)

Iter=0, Condition=-4.381757756645589e-11
Iter=1, Condition=6.458103747782828e-08
Iter=2, Condition=1.232372087876388e-07
Iter=3, Condition=1.8189337978454018e-07
Iter=4, Condition=2.4054955046853175e-07
Iter=5, Condition=2.9920572083961334e-07
Iter=6, Condition=3.5786189089778566e-07
Iter=7, Condition=4.165180606430486e-07
Iter=8, Condition=4.751742300753999e-07
Iter=9, Condition=5.33830399194842e-07
Iter=10, Condition=5.92486568001376e-07
Iter=11, Condition=6.51142736494999e-07
Iter=12, Condition=7.097989046757114e-07
Iter=13, Condition=7.68455072543518e-07
Iter=14, Condition=8.271112400984097e-07
Iter=15, Condition=8.857674073403944e-07
Iter=16, Condition=9.444235742694685e-07
Iter=17, Condition=1.0030797408856299e-06
Iter=18, Condition=1.061735907188888e-06
Iter=19, Condition=1.1203920731792322e-06
Iter=20, Condition=1.179048238856667e-06
Iter=21, Condition=1.2377044042211966e-06
Iter=22, Condition=1.2963605692728081e-06
Iter=23, Condition=1.3550167340115132e-06
Iter=24, Condition=1

KeyboardInterrupt: ignored

In [ ]:
grads@g.reshape(-1, 1)

array([[6.00513780e+09],
       [6.00340256e+09],
       [5.99555986e+09],
       [5.99320420e+09],
       [6.00620423e+09],
       [5.99949628e+09],
       [6.00267664e+09],
       [5.99861158e+09],
       [6.00229279e+09],
       [6.00078926e+09],
       [6.00217191e+09],
       [6.00203986e+09],
       [5.98433271e+09],
       [6.00589463e+09],
       [5.99776331e+09],
       [5.99630665e+09],
       [5.99649633e+09],
       [6.00296794e+09],
       [6.00620817e+09],
       [5.99847064e+09]])

### Some new implementation

In [ ]:
def null(A):
    from scipy.linalg import null_space
    n = null_space(A, rcond=1)
    if n.shape[1] < 1:
        return False, None
    r = np.sqrt(np.sum((A@n)**2, axis=0)/n.shape[1])
    if np.min(r) >= 1e-2:
        return False, None
    return True, n.T[np.argmin(r)].reshape(-1, 1)

In [ ]:
grads = 100*np.random.randn(5, 2000)+50

In [ ]:
from scipy.linalg import null_space
A = grads@grads.T
n = null_space(A, rcond=1e-1)
print(n)

[]


In [ ]:
A@n[:,]

array([  3157800.39433858,   9538843.32130915,  -6404295.86456061,
       -13290386.02744448,   7239592.15519464])

In [ ]:
null_space(grads.T, rcond=1e-1)

array([], shape=(5, 0), dtype=float64)